# Collaborators Whereabouts
Author: Sam Eriksen

Description: Notebook to read in collaboration list from google sheets and plot

In [1]:
import numpy as np
import pandas as pd
#import tqdm
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

Get author list information

In [2]:
spreadsheet_id = "1J-8ehKgEcpmssEZ_dGRIp5lRQCg6tDonYYG87Rv__4I"
sheet_id = "951549455"
sheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=csv&gid={sheet_id}"

df = pd.read_csv(sheet_url)

cut_down_df = df.loc[4:].dropna(subset=["Unnamed: 2"])

final_info = cut_down_df[["Unnamed: 47", "Unnamed: 43", "# considered (x-check)", "PRIMARY PAGE", "Unnamed: 3"]]

final_info.columns = ["Title", "Initials", "LastName", "University", "isAuthor"]

In [3]:
final_info

,Title,Initials,LastName,University,isAuthor
6,Faculty,B. J.,Mount,Black Hills State University,TRUE
7,Undergrad,A.,Rodriguez,Black Hills State University,FALSE
9,Grad Stud,L.,Korley,Brandeis University,FALSE
10,Faculty,B.,Penning,Brandeis University,FALSE
11,Post Doc,J.J.,Wang,Brandeis University,FALSE
...,...,...,...,...,...
832,Engineer,W. T.,Emmet,Yale University,FALSE
833,Technician,T. W.,Hurteau,Yale University,FALSE
834,Grad Stud,N. A.,Larsen,Yale University,FALSE
835,Grad Stud,B. P.,Tennyson,Yale University,FALSE


Get university address information

In [78]:
spreadsheet_id = "1J-8ehKgEcpmssEZ_dGRIp5lRQCg6tDonYYG87Rv__4I"
sheet_id = "1098381277"
sheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=csv&gid={sheet_id}"

df = pd.read_csv(sheet_url)

institutions = df[["SECONDARY PAGE", "Unnamed: 7"]]
institutions.columns = ["University", "Address"]
institutions = institutions.iloc[4:]
institutions = institutions.dropna(subset=['Address'])
institutions['Address'] = institutions['Address'].str.replace('GBR', 'UK')
institutions['Universities_FullName'] = institutions['University'] + ', ' + institutions['Address']
institutions['Short_Address'] = institutions['Address'].apply(lambda x: ', '.join(x.split(', ')[-2:]))
institutions['Corrected_Name'] = institutions['University'].apply(lambda x: x.split(' (')[0])
institutions['Start_Address'] = institutions['Address'].apply(lambda x: ', '.join(x.split(', ')[:2]))

In [79]:
institutions

,University,Address,Universities_FullName,Short_Address,Corrected_Name,Start_Address
4,Black Hills State University,"School of Natural Sciences, 1200 University St...","Black Hills State University, School of Natura...","SD 57799-0002, USA",Black Hills State University,"School of Natural Sciences, 1200 University St..."
5,Brookhaven National Laboratory (BNL),"P.O. Box 5000, Upton, NY 11973-5000, USA","Brookhaven National Laboratory (BNL), P.O. Box...","NY 11973-5000, USA",Brookhaven National Laboratory,"P.O. Box 5000, Upton"
6,Brandeis University,"Department of Physics, 415 South Street, Walt...","Brandeis University, Department of Physics, 4...","MA 02453, USA",Brandeis University,"Department of Physics, 415 South Street"
7,Brown University,"Department of Physics, 182 Hope Street, Provid...","Brown University, Department of Physics, 182 H...","RI 02912-9037, USA",Brown University,"Department of Physics, 182 Hope Street"
8,Center for Underground Physics (CUP),"IBS Center for Underground Physics 70, Yuseong...","Center for Underground Physics (CUP), IBS Cent...","Daejeon, KOR",Center for Underground Physics,"IBS Center for Underground Physics 70, Yuseong..."
9,Fermi National Accelerator Laboratory (FNAL),"P.O. Box 500, Batavia, IL 60510-5011, USA","Fermi National Accelerator Laboratory (FNAL), ...","IL 60510-5011, USA",Fermi National Accelerator Laboratory,"P.O. Box 500, Batavia"
10,Imperial College London,"Physics Department, Blackett Laboratory, Londo...","Imperial College London, Physics Department, B...","London SW7 2AZ, UK",Imperial College London,"Physics Department, Blackett Laboratory"
11,Kavli Institute for Particle Astrophysics and ...,"452 Lomita Mall, Stanford, CA 94305-4085 USA",Kavli Institute for Particle Astrophysics and ...,"Stanford, CA 94305-4085 USA",Kavli Institute for Particle Astrophysics and ...,"452 Lomita Mall, Stanford"
12,Laboratório de Instrumentação e Física Experim...,"Department of Physics, University of Coimbra, ...",Laboratório de Instrumentação e Física Experim...,"Coimbra, Portugal",Laboratório de Instrumentação e Física Experim...,"Department of Physics, University of Coimbra"
13,Lawrence Berkeley National Laboratory (LBNL),"1 Cyclotron Road, Berkeley, CA 94720-8099, USA","Lawrence Berkeley National Laboratory (LBNL), ...","CA 94720-8099, USA",Lawrence Berkeley National Laboratory,"1 Cyclotron Road, Berkeley"


Plot on a map

In [102]:
def get_latitude(location):
    try:
        return location.latitude
    except AttributeError:
        return None

def get_longitude(location):
    try:
        return location.longitude
    except AttributeError:
        return None
def correct_none(entry):
    long = None
    lat = None
    print(entry['Short_Address'])
    if 'Daejeon' in entry['Short_Address']:
        lat = 36.375394
        long = 127.384520
    elif '57754-1700' in entry['Short_Address']:
        lat = 44.345992
        long = -103.755154
    print(long)
    print(lat)
    return long, lat

In [81]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="university_geocoder")
# To avoid hitting the service rate limits
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [84]:
institutions['location'] = institutions['University'].apply(geocode)
institutions.loc[institutions['location'].isna(), 'location'] = institutions[institutions['location'].isna()]['Corrected_Name'].apply(geocode)
institutions.loc[institutions['location'].isna(), 'location'] = institutions[institutions['location'].isna()]['Short_Address'].apply(geocode)
institutions.loc[institutions['location'].isna(), 'location'] = institutions[institutions['location'].isna()]['Address'].apply(geocode)
institutions.loc[institutions['location'].isna(), 'location'] = institutions[institutions['location'].isna()]['Start_Address'].apply(geocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Laboratório de Instrumentação e Física Experimental de Partículas (LIP)',), **{}).
Traceback (most recent call last):
  File "/home/sam/lz/sustainable-lz/venv/lib/python3.10/site-packages/urllib3/connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
  File "/home/sam/lz/sustainable-lz/venv/lib/python3.10/site-packages/urllib3/connection.py", line 464, in getresponse
    httplib_response = super().getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1303, in recv_i

In [87]:
institutions['longitude'] = institutions['location'].apply(get_longitude)
institutions['latitude'] = institutions['location'].apply(get_latitude)

In [105]:
institutions[['longitude', 'latitude']] = institutions.apply(lambda row: correct_none(row) if pd.isna(row['latitude']) else (row['longitude'], row['latitude']), axis=1, result_type='expand')

Daejeon, KOR
127.38452
36.375394
SD 57754-1700, USA
-103.755154
44.345992


In [106]:
institutions

,University,Address,Universities_FullName,Short_Address,Corrected_Name,Start_Address,location,longitude,latitude
4,Black Hills State University,"School of Natural Sciences, 1200 University St...","Black Hills State University, School of Natura...","SD 57799-0002, USA",Black Hills State University,"School of Natural Sciences, 1200 University St...","(Black Hills State University, 1200, Universit...",-103.872854,44.498150
5,Brookhaven National Laboratory (BNL),"P.O. Box 5000, Upton, NY 11973-5000, USA","Brookhaven National Laboratory (BNL), P.O. Box...","NY 11973-5000, USA",Brookhaven National Laboratory,"P.O. Box 5000, Upton","(BNL Police, Upton Road, Brookhaven National L...",-72.887100,40.864767
6,Brandeis University,"Department of Physics, 415 South Street, Walt...","Brandeis University, Department of Physics, 4...","MA 02453, USA",Brandeis University,"Department of Physics, 415 South Street","(Brandeis University, 415, South Street, West ...",-71.258020,42.366509
7,Brown University,"Department of Physics, 182 Hope Street, Provid...","Brown University, Department of Physics, 182 H...","RI 02912-9037, USA",Brown University,"Department of Physics, 182 Hope Street","(Brown University, Eddy Street, Downtown, Prov...",-71.408801,41.818640
8,Center for Underground Physics (CUP),"IBS Center for Underground Physics 70, Yuseong...","Center for Underground Physics (CUP), IBS Cent...","Daejeon, KOR",Center for Underground Physics,"IBS Center for Underground Physics 70, Yuseong...",None,127.384520,36.375394
9,Fermi National Accelerator Laboratory (FNAL),"P.O. Box 500, Batavia, IL 60510-5011, USA","Fermi National Accelerator Laboratory (FNAL), ...","IL 60510-5011, USA",Fermi National Accelerator Laboratory,"P.O. Box 500, Batavia","(Fermi National Accelerator Laboratory, Wilson...",-88.245782,41.842258
10,Imperial College London,"Physics Department, Blackett Laboratory, Londo...","Imperial College London, Physics Department, B...","London SW7 2AZ, UK",Imperial College London,"Physics Department, Blackett Laboratory","(Imperial College London, Exhibition Road, Kni...",-0.175641,51.498959
11,Kavli Institute for Particle Astrophysics and ...,"452 Lomita Mall, Stanford, CA 94305-4085 USA",Kavli Institute for Particle Astrophysics and ...,"Stanford, CA 94305-4085 USA",Kavli Institute for Particle Astrophysics and ...,"452 Lomita Mall, Stanford","(Physics and Astrophysics, 452, Lomita Mall, S...",-122.172673,37.428226
12,Laboratório de Instrumentação e Física Experim...,"Department of Physics, University of Coimbra, ...",Laboratório de Instrumentação e Física Experim...,"Coimbra, Portugal",Laboratório de Instrumentação e Física Experim...,"Department of Physics, University of Coimbra","(Coimbra, Portugal, (40.2111931, -8.4294632))",-8.429463,40.211193
13,Lawrence Berkeley National Laboratory (LBNL),"1 Cyclotron Road, Berkeley, CA 94720-8099, USA","Lawrence Berkeley National Laboratory (LBNL), ...","CA 94720-8099, USA",Lawrence Berkeley National Laboratory,"1 Cyclotron Road, Berkeley","(Lawrence Berkeley National Laboratory, Centen...",-122.245630,37.876959


Plot on a map all universities involved